# Author: Seunghee Kim
Created on: 24.12.08

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from PIL import Image
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image

In [ ]:
class CFG:
    MODEL = "kandinsky-community/kandinsky-2-2-decoder-inpaint"
    CACHE_DIR = "./cache_dir"
    
pipeline = AutoPipelineForInpainting.from_pretrained(
    CFG.MODEL, 
    torch_dtype=torch.float16,
    cache_dir=CFG.CACHE_DIR
)
pipeline.enable_model_cpu_offload()
pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
# 예시 샘플
image_path = "./two-people-sea.jpg"        # 원본 이미지
mask_path = "./two-people-sea-mask.png"   # 마스크 이미지

# 이미지 크기 (512, 512)로 Resize
target_size = (512, 512)

def load_and_resize_image(image_path, target_size, save_path=None):
    image = load_image(image_path)
    resized_image = image.resize(target_size, Image.Resampling.LANCZOS)  
    if save_path: 
        resized_image.save(save_path)
    return resized_image


resized_original_path = "./resized-two-people-sea.jpg"
resized_mask_path = "./resized-two-people-sea-mask.png"

original_image = load_and_resize_image(image_path, target_size, resized_original_path)
mask_image = load_and_resize_image(mask_path, target_size, resized_mask_path)

result = pipeline(
    image=original_image, 
    mask_image=mask_image,
    prompt="Fill the missing area with the sea background" 

).images[0]

result_resized_path = "./two-people-sea-inpainted.jpg"
result = result.resize(target_size, Image.Resampling.LANCZOS)
result.save(result_resized_path)  

print(f"Resized original image: {resized_original_path}")
print(f"Resized mask image: {resized_mask_path}")
print(f"Inpainted result: {result_resized_path}")